In [ ]:
# python -m venv policyengine_venv
# source policyengine_venv/bin/activate
# make sure to select kernel to be in the above environment
# Install ipykernel if not already installed
!pip install ipykernel
!pip install python-dotenv
# Register the environment as a kernel
!python -m ipykernel install --user --name=policyengine_venv --display-name="Python (PolicyEngine)"
# import sys
# print(sys.executable)  # Should show a path inside your policyengine_venv directory
#Install policy engine
!pip install policyengine-uk

In [ ]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Access the token
token = os.environ.get("HUGGING_FACE_TOKEN")

# Now you can use the token
print(f"Token loaded: {token[:4]}...{token[-4:]}")  # Prints first and last 4 chars for verification

In [ ]:
from policyengine_uk import Microsimulation
from policyengine_uk.model_api import *
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
import pandas as pd
# LOCAL_enhanced_frs_2022_23="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5"
# sim = Microsimulation(dataset=LOCAL_enhanced_frs_2022_23)
sim = Microsimulation(dataset="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5")
 

In [ ]:
# Level 1: 20 GBP pw uplift to child element of UC
def change_tax_parameters(parameters):
    parameters.gov.dwp.universal_credit.elements.child.amount.update(
        start=instant("2026-01-01"), value=292.81+80.00
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", 2026).sum() - baseline.calc("gov_spending", 2026).sum()
f"Cost: £{round(revenue / 1e+9, 1)}bn"

In [ ]:
#Level2:  Work allowance increased to 1000 GBP  pa
def change_tax_parameters(parameters):
    latest_value = parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing["2025-04-01"]
    parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing.update(
        start=instant("2026-01-01"), value=83.33
    )
    latest_value = parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing["2025-04-01"]
    parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing.update(
        start=instant("2026-01-01"), value=83.33
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", 2026).sum() - baseline.calc("gov_spending", 2026).sum()
f"Cost: £{round(revenue / 1e+9, 1)}bn"

In [ ]:
#Level 2: Change child limit
def change_tax_parameters(parameters):
    parameters.gov.dwp.universal_credit.elements.child.limit.child_count.update(
        start=instant("2026-01-01"), value=inf
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", 2026).sum() - baseline.calc("gov_spending", 2026).sum()
f"Cost: £{round(revenue / 1e+9, 1)}bn"

In [ ]:
from policyengine_uk import Microsimulation
from policyengine_uk.reforms.uc_bonus_for_under_5 import reform

baseline = Microsimulation()
reform = AddUCChildUnder5Bonus()
reformed = Microsimulation(reform=reform)

reformed.calculate("universal_credit", "2025-05")

In [ ]:
from policyengine_uk import Microsimulation
from policyengine_uk.reforms.uc_bonus_for_under5 import reform
sim = Microsimulation()
sim.tax_benefit_system.parameters("2025-04-01").gov.dwp.universal_credit.elements.child.uc_individual_child_element

In [ ]:
from policyengine_uk import Microsimulation
from policyengine_uk.reforms.uc_bonus_for_under5 import reform
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
print(reformed.calculate("num_households_with_under5", "2025").sum())
print(reformed.calculate("universal_credit", "2025").mean())
print(baseline.calculate("universal_credit", "2025").mean())

revenue = reformed.calculate("gov_spending", 2025).sum() - baseline.calc("gov_spending", 2025).sum()
print(f"Revenue: £{revenue / 1e9:.1f}bn")
under5_households = reformed.calculate("num_households_with_under5", "2025-05").sum()
print(f"Number of households with at least one child under 5: {under5_households}")

3328869.458897898
1885.3390008208485
1877.0911343386936
Revenue: £-172844578
Number of households with at least one child under 5: 277405.79650882824


In [ ]:
HOUSEHOLD_VARIABLES = ["person_id", "household_id", "age", "household_net_income", "household_income_decile", "in_poverty", "household_tax", "household_benefits"]
baseline_person_df = baseline.calculate_dataframe(HOUSEHOLD_VARIABLES, 2026).astype(float)
reformed_person_df = reformed.calculate_dataframe(HOUSEHOLD_VARIABLES, 2026).astype(float)
difference_person_df = reformed_person_df - baseline_person_df
total_net_income_baseline = baseline.calculate("household_net_income", 2026).sum()
total_net_income_reformed = reformed.calculate("household_net_income", 2026).sum()

net_cost = total_net_income_reformed - total_net_income_baseline

print(f"This reform would cost £{net_cost / 1e9:.1f}bn")

In [ ]:

def modify_parameters(parameters):
    parameters.gov.dwp.universal_credit.standard_allowance.amount.COUPLE_OLD.update(start=instant("2024-01-01"), value=738.82)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.COUPLE_YOUNG.update(start=instant("2024-01-01"), value=618.51)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.SINGLE_OLD.update(start=instant("2024-01-01"), value=528.74)
    parameters.gov.dwp.universal_credit.standard_allowance.amount.SINGLE_YOUNG.update(start=instant("2024-01-01"), value=452.11)
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters)

baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", 2025).sum() - baseline.calc("gov_spending", 2025).sum()
f"Revenue: £{round(revenue / 1e+9, 1)}bn"